<a href="https://colab.research.google.com/github/PilorumNatum/IA_languesAnciennes/blob/code/Code/Unsupervised_reconstruction/ProtoFormRecontruction_Experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Approche neuronale non supervisée pour la reconstruction de proto-formes.

Auteurs : Benjamin BADOUAILLE, Eliott CAMOU, Thomas HORRUT


---


*Ce notebook contient une tentative d'implémentation de l'intelligence artificielle décrite par un article de [Andre He et al.](https://arxiv.org/abs/2211.08684) pour prédire l'état phonétique de proto-forms associés à un ensemble de cognats. L'expérience est ici menée dans le cadre d'un projet de recherche scientifique de fin de cycle préparatoire, dont l'analyse des démarches employées et des résultats nourrira une réflexion plus globale sur le potentiel de l'IA pour résoudre des problèmes de Linguistique Historique.*

### Installation des dépendances.

In [ ]:
!pip3 install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch

torch.rand(2,3)

tensor([[0.5993, 0.5377, 0.2959],
        [0.8726, 0.2115, 0.4264]])

# Définition des données associées au cadre de travail



*   $L$ est l'ensemble des langues romanes avec lesquelles nous travaillons.
*   $\Sigma$ est l'ensemble des caractères IPA composant les proto-mots et les cognats. La liste a été générée à partir du [site de l'IPA](https://www.internationalphoneticalphabet.org/ipa-charts/ipa-symbols-with-unicode-decimal-and-hex-codes/).
*   La liste $C$ des ensembles de cognats. 



## Initialisation de $L$

In [ ]:
L = {"French", "Italian", "Spanish", "Portuguese"}

## Initialisation de $Σ$

À chaque réinstanciation de l'environnement d'exécution Google Colab, importer le fichier `IPA_characters.csv` situé sur le repo Github (dans le dossier `./Code/`; pour l'instant il n'est que dans la branche `code` du repo).

In [64]:
from google.colab import files
import csv

EXPECTED_FILENAME = "IPA_characters.csv"
EXPECTED_FIELD_NAMES = {"symbol", "decimal", "hex", "value", "type"}
EXPECTED_IPA_CHARACTERS_NUMBER = 138


#Check if the file has already been uploaded
content = !ls /content
file_already_exist = EXPECTED_FILENAME in content[0].split(' ')
if not file_already_exist:
  #If it has not, then an upload is requested
  print("Veuillez déposer le fichier 'IPA_characters.csv':")
  uploaded = files.upload()
  if "IPA_characters.csv" in uploaded:
   raise Exception("Le fichier importé ne correspond au fichier attendu")

IPA_CHARACTERS:list[str] = []
with open(EXPECTED_FILENAME, 'r', encoding='UTF32') as f:
  reader = csv.DictReader(f)
  if EXPECTED_FIELD_NAMES != set(reader.fieldnames):
    raise Exception("Les entrées du fichier csv sont mal définies.")
  i = 0
  for row in reader:
    IPA_CHARACTERS.append(row['symbol'])
    i+=1
  if i!=EXPECTED_IPA_CHARACTERS_NUMBER:
    raise Exception(
        "Le nombre de caractères IPA dans ce fichier est incorrect")

print('Liste des caractères IPA importée :')
for i in range(len(IPA_CHARACTERS)):
  print(f"'{IPA_CHARACTERS[i]}'", end=(' ', '\n')[i%21==20])
print()

Liste des caractères IPA importée :
'ɑ' 'ɐ' 'ɒ' 'æ' 'ɓ' 'ʙ' 'β' 'ɔ' 'ɕ' 'ç' 'ɗ' 'ɖ' 'ð' 'ʤ' 'ə' 'ɘ' 'ɚ' 'ɛ' 'ɜ' 'ɝ' 'ɞ'
'ɟ' 'ʄ' 'ɡ' 'ɠ' 'ɢ' 'ʛ' 'ɦ' 'ɧ' 'ħ' 'ɥ' 'ʜ' 'ɨ' 'ɪ' 'ʝ' 'ɭ' 'ɬ' 'ɫ' 'ɮ' 'ʟ' 'ɱ' 'ɯ'
'ɰ' 'ŋ' 'ɳ' 'ɲ' 'ɴ' 'ø' 'ɵ' 'ɸ' 'θ' 'œ' 'ɶ' 'ʘ' 'ɹ' 'ɺ' 'ɾ' 'ɻ' 'ʀ' 'ʁ' 'ɽ' 'ʂ' 'ʃ'
'ʈ' 'ʧ' 'ʉ' 'ʊ' 'ʋ' 'ⱱ' 'ʌ' 'ɣ' 'ɤ' 'ʍ' 'χ' 'ʎ' 'ʏ' 'ʑ' 'ʐ' 'ʒ' 'ʔ' 'ʡ' 'ʕ' 'ʢ' 'ǀ'
'ǁ' 'ǂ' 'ǃ' 'ˈ' 'ˌ' 'ː' 'ˑ' 'ʼ' 'ʴ' 'ʰ' 'ʱ' 'ʲ' 'ʷ' 'ˠ' 'ˤ' '˞' 'n̥ d̥' 'ŋ̊' 'b̤ a̤' 't̪ d̪' 's̬ t̬'
'b̰ a̰' 't̺ d̺' 't̼ d̼' 't̻ d̻' 't̚' 'ɔ̹' 'ẽ' 'ɔ̜' 'u̟' 'e̠' 'ë' 'l̴ n̴' 'ɫ' 'e̽' 'e̝ ɹ̝' 'm̩ n̩ l' 'e̞ β̞' 'e̯' 'e̘' 'e̙' 'ĕ'
'e̋' 'é' 'ē' 'è' 'ȅ' 'x͜' 'x͡' '↓' '↑' '→' '' '' 
